In [1]:
import pickle as pkl
import numpy as np
import pandas as pd
import sklearn
from twitter_nlp_toolkit.tweet_sentiment_classifier import tweet_sentiment_classifier
from twitter_nlp_toolkit.file_fetcher import file_fetcher
from zipfile import ZipFile

Using TensorFlow backend.


In [2]:
chunk = 1 # Fraction of data to train on - you can reduce for debugging for speed
redownload=False
model_path = 'Models'


Here we download pre-trained models and a validation dataset. The models have been trained on the Sentiment140 dataset, taken form here: https://www.kaggle.com/kazanova/sentiment140

The validation data is hand-labeled airline customer feedback taken from https://www.figure-eight.com/data-for-everyone/

In [3]:
if redownload: 
    downloader = file_fetcher.downloader(using_notebook=True)
    # Validation data
    downloader.download_file('https://www.dropbox.com/s/440m6x07bjg6c0h/Tweets.zip?dl=1',"Tweets.zip")
    
    # Compressed model
    # Note that this model has only been trained on 5% of the training data
    # Update when better-trained model is available
    downloader.download_file("https://www.dropbox.com/s/i88eqlja56xncyx/model_test_05.zip?dl=1","Models.zip")
    
    # Extract all the contents of zip file in current directory
    with ZipFile('Models.zip', 'r') as zipObj:
        zipObj.extractall(path=model_path)

In [4]:
# Load the validation data

test_data = pd.read_csv('Tweets.zip', header=0, names=['Index', 'Sentiment', 'Sentiment_confidence',
                                                                'Negative_reason', 'Negative_reason_confidence',
                                                                'Airline', 'Airline_sentiment_gold', 'Handle',
                                                                'Negative_reason_gold', 'Retweet_count', 'Text',
                                                                'Tweet_coord', 'Time', 'Location', 'Timezone'])

In [5]:
# Remove the unlabeled test data

test_data['Labels'] = (test_data['Sentiment'] == 'positive') * 2
test_data['Labels'] = test_data['Labels'] + (test_data['Sentiment'] == 'neutral') * 1
test_data['Labels'] = test_data['Labels'] / 2

test_data.set_index('Labels')
test_data = test_data[test_data.Labels != 0.5]

In [6]:
# Executing this cell takes about 30s on a laptop

Classifier = tweet_sentiment_classifier.SentimentAnalyzer()
Classifier.load_models(path=model_path)

BoW model Models\bow loaded successfully
BoW model Models\bow_005_1 loaded successfully
BoW model Models\bow_005_2 loaded successfully
BoW model Models\bow_005_3 loaded successfully
Pre-trained embedding model loaded successfully
Pre-trained embedding model loaded successfully
Pre-trained embedding model loaded successfully
Pre-trained embedding model loaded successfully
LSTM model Models\lstm loaded successfully
LSTM model Models\lstm_005_1 loaded successfully
LSTM model Models\lstm_005_2 loaded successfully
LSTM model Models\lstm_005_3 loaded successfully


We santiy check the models: 

In [7]:
Classifier.predict(['I am happy', 'I am sad', 'I am cheerful', 'I am mad'])


array([1., 0., 1., 0.])

We test the model on an airline customer feedback dataset.

In [8]:
# Executing this cell takes several minuites on a laptop

predictions = Classifier.predict(test_data['Text'])


In [9]:
print('Test Accuracy:  {:.3f}'.format(sklearn.metrics.accuracy_score(test_data['Labels'], predictions)))
print('Test MCC:  {:.3f}'.format(sklearn.metrics.matthews_corrcoef(test_data['Labels'], predictions)))
sklearn.metrics.confusion_matrix(test_data['Labels'], predictions)

Test Accuracy:  0.759
Test MCC:  0.512


array([[6634, 2544],
       [ 235, 2128]], dtype=int64)

We have accuracy of just under 80%.


We split our evaluation dataset into validation and testing and eliminate the worst-performing models

In [10]:
valX, testX, valY, testY = sklearn.model_selection.train_test_split(test_data['Text'], test_data['Labels'], test_size=0.5, stratify=test_data['Labels'])

In [11]:
# Executing this cell takes several minuites on a laptop


Classifier.trim_models(valX, valY, threshold=0.7)

Model Models\bow score: 0.801
Model Models\bow_005_1 score: 0.741
Model Models\bow_005_2 score: 0.759
Model Models\bow_005_3 score: 0.750
Model Models\glove score: 0.771
Model Models\glove_005_1 score: 0.726
Model Models\glove_005_2 score: 0.673
Deleting model Models\glove_005_2
Model Models\glove_005_3 score: 0.679
Deleting model Models\glove_005_3
Model Models\lstm score: 0.686
Deleting model Models\lstm
Model Models\lstm_005_1 score: 0.653
Deleting model Models\lstm_005_1
Model Models\lstm_005_2 score: 0.642
Deleting model Models\lstm_005_2
Model Models\lstm_005_3 score: 0.648
Deleting model Models\lstm_005_3


Our custom-embedding models performed poorly on this dataset and have been pruned. 

In [12]:
predictions = Classifier.predict(testX)

print('Test Accuracy:  {:.3f}'.format(sklearn.metrics.accuracy_score(testY, predictions)))
print('Test MCC:  {:.3f}'.format(sklearn.metrics.matthews_corrcoef(testY, predictions)))
sklearn.metrics.confusion_matrix(testY, predictions)

Test Accuracy:  0.773
Test MCC:  0.517


array([[3425, 1164],
       [ 147, 1035]], dtype=int64)

Pruning our models had a minor impact on our classification performance, bringing us to an acceptible ~77%. 

To improve our accuracy, we can refine the models on our airline data. The early stopping procedure (enabled by default to use 20% of the training data for validation) should minimize overfitting.

In [13]:
Classifier.refine(valX, valY)
test_predictions = Classifier.predict(testX)

predictions = Classifier.predict(testX)

Train on 4616 samples, validate on 1154 samples
Epoch 1/500
4616/4616 [==============================] - 2s 528us/step - loss: 0.4158 - acc: 0.8217 - val_loss: 0.3132 - val_acc: 0.8648
Epoch 2/500
4616/4616 [==============================] - 2s 367us/step - loss: 0.3568 - acc: 0.8546 - val_loss: 0.2758 - val_acc: 0.8873
Epoch 3/500
4616/4616 [==============================] - 2s 377us/step - loss: 0.3079 - acc: 0.8795 - val_loss: 0.2531 - val_acc: 0.8986
Epoch 4/500
4616/4616 [==============================] - 2s 365us/step - loss: 0.2955 - acc: 0.8871 - val_loss: 0.2548 - val_acc: 0.8995
Epoch 5/500
4616/4616 [==============================] - 2s 382us/step - loss: 0.2895 - acc: 0.8954 - val_loss: 0.2557 - val_acc: 0.8995
Epoch 6/500
4616/4616 [==============================] - 2s 349us/step - loss: 0.2989 - acc: 0.8941 - val_loss: 0.2595 - val_acc: 0.9012
Epoch 7/500
4616/4616 [==============================] - 2s 350us/step - loss: 0.3065 - acc: 0.9019 - val_loss: 0.2640 - val_acc: 

Epoch 60/500
4616/4616 [==============================] - 2s 367us/step - loss: 0.2076 - acc: 0.9218 - val_loss: 0.1958 - val_acc: 0.9229
Epoch 61/500
4616/4616 [==============================] - 2s 357us/step - loss: 0.2102 - acc: 0.9188 - val_loss: 0.2030 - val_acc: 0.9229
Epoch 62/500
4616/4616 [==============================] - 2s 354us/step - loss: 0.2198 - acc: 0.9153 - val_loss: 0.2032 - val_acc: 0.9220
Epoch 63/500
4616/4616 [==============================] - 2s 355us/step - loss: 0.2046 - acc: 0.9190 - val_loss: 0.2094 - val_acc: 0.9220
Epoch 64/500
4616/4616 [==============================] - 2s 351us/step - loss: 0.2061 - acc: 0.9203 - val_loss: 0.2084 - val_acc: 0.9229
Epoch 65/500
4616/4616 [==============================] - 2s 354us/step - loss: 0.2066 - acc: 0.9205 - val_loss: 0.2078 - val_acc: 0.9229
Epoch 66/500
4616/4616 [==============================] - 2s 358us/step - loss: 0.2040 - acc: 0.9175 - val_loss: 0.2072 - val_acc: 0.9229
Epoch 67/500
4616/4616 [==========

4616/4616 [==============================] - 2s 381us/step - loss: 0.1867 - acc: 0.9276 - val_loss: 0.1958 - val_acc: 0.9298
Epoch 120/500
4616/4616 [==============================] - 2s 375us/step - loss: 0.1925 - acc: 0.9218 - val_loss: 0.1959 - val_acc: 0.9281
Epoch 121/500
4616/4616 [==============================] - 2s 380us/step - loss: 0.1919 - acc: 0.9274 - val_loss: 0.1962 - val_acc: 0.9281
Epoch 122/500
4616/4616 [==============================] - 2s 379us/step - loss: 0.1924 - acc: 0.9244 - val_loss: 0.1966 - val_acc: 0.9272
Epoch 123/500
4616/4616 [==============================] - 2s 389us/step - loss: 0.1902 - acc: 0.9261 - val_loss: 0.1929 - val_acc: 0.9272
Epoch 124/500
4616/4616 [==============================] - 2s 376us/step - loss: 0.1891 - acc: 0.9276 - val_loss: 0.1906 - val_acc: 0.9255
Epoch 125/500
4616/4616 [==============================] - 2s 372us/step - loss: 0.1884 - acc: 0.9253 - val_loss: 0.1905 - val_acc: 0.9255
Epoch 126/500
4616/4616 [================

Epoch 178/500
4616/4616 [==============================] - 2s 358us/step - loss: 0.1684 - acc: 0.9333 - val_loss: 0.1901 - val_acc: 0.9298
Epoch 179/500
4616/4616 [==============================] - 2s 389us/step - loss: 0.1681 - acc: 0.9287 - val_loss: 0.1917 - val_acc: 0.9281
Epoch 180/500
4616/4616 [==============================] - 2s 386us/step - loss: 0.1702 - acc: 0.9322 - val_loss: 0.1934 - val_acc: 0.9289
Epoch 181/500
4616/4616 [==============================] - 2s 391us/step - loss: 0.1670 - acc: 0.9328 - val_loss: 0.1998 - val_acc: 0.9289
Epoch 182/500
4616/4616 [==============================] - 2s 406us/step - loss: 0.1755 - acc: 0.9298 - val_loss: 0.1916 - val_acc: 0.9298
Epoch 183/500
4616/4616 [==============================] - 2s 377us/step - loss: 0.1645 - acc: 0.9344 - val_loss: 0.1899 - val_acc: 0.9298
Epoch 184/500
4616/4616 [==============================] - 2s 351us/step - loss: 0.1625 - acc: 0.9337 - val_loss: 0.1888 - val_acc: 0.9298
Epoch 185/500
4616/4616 [==

Epoch 237/500
4616/4616 [==============================] - 2s 345us/step - loss: 0.1646 - acc: 0.9346 - val_loss: 0.1804 - val_acc: 0.9333
Epoch 238/500
4616/4616 [==============================] - 2s 344us/step - loss: 0.1682 - acc: 0.9367 - val_loss: 0.1805 - val_acc: 0.9324
Epoch 239/500
4616/4616 [==============================] - 2s 346us/step - loss: 0.1627 - acc: 0.9335 - val_loss: 0.1812 - val_acc: 0.9315
Epoch 240/500
4616/4616 [==============================] - 2s 360us/step - loss: 0.1576 - acc: 0.9370 - val_loss: 0.1821 - val_acc: 0.9315
Epoch 241/500
4616/4616 [==============================] - 2s 367us/step - loss: 0.1566 - acc: 0.9374 - val_loss: 0.1803 - val_acc: 0.9333
Epoch 242/500
4616/4616 [==============================] - 2s 383us/step - loss: 0.1584 - acc: 0.9402 - val_loss: 0.1805 - val_acc: 0.9350
Epoch 243/500
4616/4616 [==============================] - 2s 367us/step - loss: 0.1617 - acc: 0.9387 - val_loss: 0.1815 - val_acc: 0.9359
Epoch 244/500
4616/4616 [==

Epoch 00295: early stopping
Train on 4616 samples, validate on 1154 samples
Epoch 1/500
4616/4616 [==============================] - 3s 542us/step - loss: 0.4413 - acc: 0.8000 - val_loss: 0.2977 - val_acc: 0.8761
Epoch 2/500
4616/4616 [==============================] - 2s 385us/step - loss: 0.3585 - acc: 0.8618 - val_loss: 0.2551 - val_acc: 0.8943
Epoch 3/500
4616/4616 [==============================] - 2s 389us/step - loss: 0.3168 - acc: 0.8819 - val_loss: 0.2661 - val_acc: 0.8925
Epoch 4/500
4616/4616 [==============================] - 2s 392us/step - loss: 0.3290 - acc: 0.8899 - val_loss: 0.3107 - val_acc: 0.8917
Epoch 5/500
4616/4616 [==============================] - 2s 400us/step - loss: 0.3737 - acc: 0.8897 - val_loss: 0.3259 - val_acc: 0.8865
Epoch 6/500
4616/4616 [==============================] - 2s 403us/step - loss: 0.3720 - acc: 0.8865 - val_loss: 0.2918 - val_acc: 0.8873
Epoch 7/500
4616/4616 [==============================] - 2s 415us/step - loss: 0.3662 - acc: 0.8899 - 

Epoch 60/500
4616/4616 [==============================] - 2s 371us/step - loss: 0.2109 - acc: 0.9166 - val_loss: 0.1901 - val_acc: 0.9246
Epoch 61/500
4616/4616 [==============================] - 2s 370us/step - loss: 0.2051 - acc: 0.9203 - val_loss: 0.1983 - val_acc: 0.9255
Epoch 62/500
4616/4616 [==============================] - 2s 394us/step - loss: 0.2089 - acc: 0.9229 - val_loss: 0.1989 - val_acc: 0.9237
Epoch 63/500
4616/4616 [==============================] - 2s 392us/step - loss: 0.2074 - acc: 0.9229 - val_loss: 0.1998 - val_acc: 0.9246
Epoch 64/500
4616/4616 [==============================] - 2s 418us/step - loss: 0.2052 - acc: 0.9190 - val_loss: 0.2007 - val_acc: 0.9237
Epoch 65/500
4616/4616 [==============================] - 2s 411us/step - loss: 0.2057 - acc: 0.9227 - val_loss: 0.2015 - val_acc: 0.9237
Epoch 66/500
4616/4616 [==============================] - 2s 386us/step - loss: 0.2085 - acc: 0.9192 - val_loss: 0.2023 - val_acc: 0.9229
Epoch 67/500
4616/4616 [==========

4616/4616 [==============================] - 2s 381us/step - loss: 0.1672 - acc: 0.9328 - val_loss: 0.2022 - val_acc: 0.9315
Epoch 120/500
4616/4616 [==============================] - 2s 387us/step - loss: 0.1711 - acc: 0.9307 - val_loss: 0.2034 - val_acc: 0.9307
Epoch 121/500
4616/4616 [==============================] - 2s 378us/step - loss: 0.1734 - acc: 0.9279 - val_loss: 0.2140 - val_acc: 0.9307
Epoch 122/500
4616/4616 [==============================] - 2s 381us/step - loss: 0.1635 - acc: 0.9324 - val_loss: 0.2275 - val_acc: 0.9289
Epoch 123/500
4616/4616 [==============================] - 2s 373us/step - loss: 0.1871 - acc: 0.9276 - val_loss: 0.2415 - val_acc: 0.9281
Epoch 124/500
4616/4616 [==============================] - 2s 373us/step - loss: 0.1692 - acc: 0.9302 - val_loss: 0.2512 - val_acc: 0.9263
Epoch 125/500
4616/4616 [==============================] - 2s 378us/step - loss: 0.1766 - acc: 0.9298 - val_loss: 0.2516 - val_acc: 0.9255
Epoch 126/500
4616/4616 [================

In [14]:
print('Test Accuracy:  {:.3f}'.format(sklearn.metrics.accuracy_score(testY, predictions)))
print('Test MCC:  {:.3f}'.format(sklearn.metrics.matthews_corrcoef(testY, predictions)))
sklearn.metrics.confusion_matrix(testY, predictions)

Test Accuracy:  0.895
Test MCC:  0.650


array([[4506,   83],
       [ 523,  659]], dtype=int64)

Now we have accuracies of nearly 90%! 

In [15]:
Classifier.evaluate(testX, testY)

Model Models\bow score: 0.891
Model Models\bow_005_1 score: 0.889
Model Models\bow_005_2 score: 0.889
Model Models\bow_005_3 score: 0.888
Model Models\glove score: 0.888
Model Models\glove_005_1 score: 0.881


{'ensembled': 0.8949922023912666,
 'Models\\bow': 0.8910067579275689,
 'Models\\bow_005_1': 0.8891006757927569,
 'Models\\bow_005_2': 0.8885808352105354,
 'Models\\bow_005_3': 0.8882342748223878,
 'Models\\glove': 0.8877144342401664,
 'Models\\glove_005_1': 0.8809565066712874}